In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import sys
import csv
from batchmake import Batcher
import astwalker
import ast
import tokenize
from io import StringIO

In [2]:
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"

In [7]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
batch_data = datas+'output.txt.part0'
# batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
    
print(len(picked_data))
train_proportion = 0.5
valid_proportion = 0.2

2515


In [8]:
subdirectories = [os.path.join(datas,o) for o in os.listdir(datas) if os.path.isdir(os.path.join(datas,o))]

In [9]:
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [10]:
train_split = int(len(python_files) * train_proportion)
valid_split = train_split + int(len(python_files) * valid_proportion)
train_files = []
valid_files = []
test_files = []

In [11]:
for project in python_files[:train_split]:
    train_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[train_split:valid_split]:
    valid_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[valid_split:]:
    test_files.extend([f[len(datas):] for f in project[1]])

In [12]:
def write_to_file(fname, lst):
    with open(os.path.join(datas, fname), "w") as write_file:
        for f in lst:
            print(f, file=write_file)

In [13]:
write_to_file("train_files.txt", train_files)
write_to_file("valid_files.txt", valid_files)
write_to_file("test_files.txt", test_files)

### 데이터셋을 train, valid, test셋으로 만든다.

### 파일 읽기

In [14]:
python_files = [os.path.join(datas, f) for f in train_files]
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\r\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

In [15]:
def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [16]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in python_files:
    try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
    except:
        error_data+=1
        continue
        print("Error when tokenizing %s: %s" % (filename, sys.exc_info()[:3]))

### 전체 파일 ( python_files ) 중에서 에러난거 뺀데이터 (data) 

In [17]:
len(python_files),error_data,len(data), len(definition_positions)

(3675, 41, 3634, 3634)

### 데이터 비교

In [18]:
d , dd=get_source_tree(data_file_name[0])
print(d)

import random
var2213 = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*(-_=+)'
var4730 = random.SystemRandom()
print(''.join(var4730.sample(var2213, 50)))



In [19]:
token_to_str = []
for tokend_data,position in data[0]:
     token_to_str.append(inv_map[tokend_data])
print(' '.join(token_to_str))

import random 
 var2213 = §OOV§ 
 var4730 = random . §OOV§ ( ) 
 print ( '' . join ( var4730 . sample ( var2213 , §NUM§ ) ) ) 



### 원본데이터와 토큰데이터

In [20]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}

identifier_types = [key for key in type_max_rands]
def_positions = [[[t[1] for t in fp if t[0] == k] for k in identifier_types] for fp in definition_positions]
# ex) def_positions[101] #function, var, class, attribute, arg

In [21]:
inputs = []
for line_data in data[:30]:
    line_data_list = []
    for token in line_data:
        line_data_list.append(token[0])
        if token[0] == 2:
            inputs.append(line_data_list)
            line_data_list=[]
            
#             print(token[0])
        

In [22]:
for row in inputs:
    print(row)

[20, 371, 2]
[4817, 7, 1, 2]
[6541, 7, 371, 5, 1, 3, 4, 2]
[282, 3, 55, 5, 102, 3, 6541, 5, 4114, 3, 4817, 6, 12, 4, 4, 4, 2]
[20, 59, 2]
[20, 219, 2]
[26, 3, 232, 56, 882, 4, 8, 2]
[10, 59, 5, 560, 5, 1213, 3, 3499, 6, 1, 4, 2]
[22, 27, 5, 133, 5, 246, 20, 8884, 2]
[8884, 3, 219, 5, 4772, 4, 2]
[22, 151, 20, 258, 2]
[22, 27, 5, 150, 20, 659, 2]
[2]
[2]
[23, 4122, 3, 659, 4, 8, 2]
[10, 3140, 7, 2415, 2]
[5175, 7, 11578, 2]
[2]
[15, 10422, 3, 9, 4, 8, 2]
[10, 20, 8934, 5, 321, 2]
[22, 151, 20, 258, 2]
[20, 445, 2]
[22, 27, 5, 44, 5, 21, 20, 143, 2]
[22, 1657, 5, 21, 20, 345, 2]
[22, 1658, 5, 390, 20, 5638, 2]
[22, 7081, 5, 21, 20, 1144, 2]
[22, 1001, 5, 390, 20, 906, 6, 4123, 2]
[22, 5, 21, 20, 3016, 6, 1846, 6, 5639, 6, 6557, 2]
[2]
[2]
[23, 3801, 3, 5638, 6, 445, 5, 3017, 4, 8, 2]
[10, 1475, 7, 4123, 3, 28, 7, 69, 6, 6068, 7, 2416, 4, 2]
[1343, 7, 445, 5, 2782, 3, 520, 7, 5182, 6, 289, 7, 6558, 4, 2]
[2327, 7, 445, 5, 1112, 3, 28, 7, 1, 6, 308, 7, 345, 5, 19, 5, 53, 3, 4, 6, 289, 7, 9

In [23]:
with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile)
    for row_line in inputs:
        spamwriter.writerow(row_line)

### target은 input+1 의 인덱스로 하고, sequence는 실행시간에 정함.